In [1]:
from pyomo.environ import *

# Initialize the model
model = ConcreteModel()


In [2]:
#from tradeups import *

#trade_up_pool = load_tradeup_4()
#trade_up_pool = load_tradeup_filler1()
#trade_up_pool.updateAllPrices()

In [3]:
from active_collections import load_active_milspec_tradeup

trade_up_pool = load_active_milspec_tradeup()
trade_up_pool.updateAllPrices()

[(0, 0.07), (0.07, 0.15), (0.15, 0.37), (0.37, 0.45), (0.45, 1)]
['Factory New', 'Minimal Wear', 'Field-Tested', 'Well-Worn', 'Battle-Scarred']
Updated skin SSG 08 | Hand Brake's price to 1.04
Updated skin SSG 08 | Hand Brake's price to 0.77
Updated skin SSG 08 | Hand Brake's price to 0.53
Updated skin SSG 08 | Hand Brake's price to 0.37
Updated skin SSG 08 | Hand Brake's price to 0.31
[(0, 0.07), (0.07, 0.15), (0.15, 0.37), (0.37, 0.4)]
['Factory New', 'Minimal Wear', 'Field-Tested', 'Well-Worn']
Updated skin M4A4 | Converter's price to 0.93
Updated skin M4A4 | Converter's price to 0.75
Updated skin M4A4 | Converter's price to 0.61
Updated skin M4A4 | Converter's price to 0.7
[(0, 0.07), (0.07, 0.08)]
['Factory New', 'Minimal Wear']
Updated skin Sawed-Off | Brake Light's price to 0.87
Updated skin Sawed-Off | Brake Light's price to 0.85
[(0, 0.07), (0.07, 0.15), (0.15, 0.37), (0.37, 0.45), (0.45, 0.6)]
['Factory New', 'Minimal Wear', 'Field-Tested', 'Well-Worn', 'Battle-Scarred']
Upda

In [4]:
# Dynamically create variables for input skins
collections = []
collection_dict = {} # maps collection name to collection. Collection name is unique
all_input_skins = []
all_output_skins = []
for collection in trade_up_pool.collections:
    input_skins = []
    output_skins = []
    for skin in collection.input_skins:
        skin_prices = skin.get_prices()
        skin_floats = skin.get_median_floats(ratio=0.2)
        num_floats = len(skin_floats)
        
        skin_count_var = Var(range(num_floats), within=NonNegativeIntegers, bounds=(0, 10))
        setattr(model, f"{skin.name}_count", skin_count_var)
        #print("sum - ", sum(skin_count_var[index] for index in skin_count_var))

        input_skins.append((skin_count_var, skin_prices, skin_floats))
        all_input_skins.append((skin_count_var, skin_prices, skin_floats))
    for skin in collection.output_skins:
        skin_name = skin.get_name()
        skin_prices = skin.get_prices()
        skin_bounded_floats = skin.get_bounded_floats()

        output_var = Var(range(len(skin_bounded_floats)), within=Binary)
        setattr(model, f"{skin_name}_output", output_var)

        output_skin = (output_var, skin_name, skin_prices, skin_bounded_floats)

        output_skins.append(output_skin)
        all_output_skins.append(output_skin)
    collections.append((input_skins, output_skins))
    collection_dict[collection.name] = (input_skins, output_skins)

for collection in collections:
    print(collection)
        

([(<pyomo.core.base.var.IndexedVar object at 0x00000238158223F0>, [1.04, 0.77, 0.53, 0.37, 0.31], [0.014000000000000002, 0.08600000000000001, 0.194, 0.386, 0.56]), (<pyomo.core.base.var.IndexedVar object at 0x00000238152872C0>, [0.93, 0.75, 0.61, 0.7], [0.014000000000000002, 0.08600000000000001, 0.194, 0.376]), (<pyomo.core.base.var.IndexedVar object at 0x0000023815A2F020>, [0.87, 0.85], [0.014000000000000002, 0.07200000000000001]), (<pyomo.core.base.var.IndexedVar object at 0x0000023815A2EB70>, [1.5, 0.87, 0.6, 0.62, 0.38], [0.014000000000000002, 0.08600000000000001, 0.194, 0.386, 0.48])], [(<pyomo.core.base.var.IndexedVar object at 0x0000023815A728D0>, 'AK-47 | Safety Net', [8.0, 5.49, 3.0, 3.79, 2.94], [(0, 0.07), (0.07, 0.15), (0.15, 0.37), (0.37, 0.45), (0.45, 0.6)]), (<pyomo.core.base.var.IndexedVar object at 0x0000023815A71220>, 'MP7 | Fade', [8.0, 5.19, 4.25], [(0, 0.07), (0.07, 0.15), (0.15, 0.25)]), (<pyomo.core.base.var.IndexedVar object at 0x0000023815A70DA0>, 'P250 | Vino 

In [5]:
# Objective: Maximize skin value (Pyomo minimizes by default, so you need to multiply by -1)
# currently the objective is the input price

# bottom part of the fraction
model.input_skins_cost = Var(domain=NonNegativeReals)

# top part of the fraction
model.average_output_price = Var(domain=NonNegativeReals)
for (skin_count_var, skin_prices, _) in all_input_skins:
    print(skin_prices)

input_skins_cost = sum(sum(skin_count_var[i] * skin_prices[i] for i in range(len(skin_prices))) for (skin_count_var, skin_prices, _) in all_input_skins)
    
def input_skins_cost_rule(model):
    return model.input_skins_cost == input_skins_cost
model.input_skins_cost_constraint = Constraint(rule=input_skins_cost_rule)

def objective_rule(model):
    return model.input_skins_cost

def new_objective_rule(model):
    return model.average_output_price / model.input_skins_cost

model.objective = Objective(rule=new_objective_rule, sense=maximize)


[1.04, 0.77, 0.53, 0.37, 0.31]
[0.93, 0.75, 0.61, 0.7]
[0.87, 0.85]
[1.5, 0.87, 0.6, 0.62, 0.38]
[0.55, 0.41, 0.2, 0.15, 0.15]
[0.58, 0.39, 0.2, 0.16, 0.14]
[0.58, 0.41, 0.2, 0.15, 0.14]
[1.88, 0.83, 0.34, 0.71, 0.31]
[1.07, 0.73, 0.37, 0.31, 0.3]
[1.63, 0.85, 0.37, 0.35, 0.31]
[1.09, 0.77, 0.55]
[2.23, 0.94, 0.53, 0.38, 0.31]
[1.22, 0.94, 0.53, 0.37, 0.31]
[1.18, 0.92, 0.58, 0.39, 0.32]
[0.76, 0.39, 0.29]
[0.53, 0.31, 0.2, 0.2, 0.17]
[0.53, 0.41]
[0.5, 0.34, 0.21, 0.2, 0.17]
[0.53, 0.31, 0.2, 0.17, 0.15]
[0.52, 0.41]
[0.98]
[0.63, 0.38, 0.21, 0.19, 0.18]
[0.61, 0.37, 0.2, 0.34]
[0.53, 0.32, 0.2, 0.19, 0.21]
[0.52, 0.31, 0.2, 0.19]


In [6]:
# Total skins constraint
def total_skins_constraint(model):
    # trade up must have 10 input skins. So the sum of all skin count variables is 10
    return sum(sum(skin_count_var[i] for i in range(len(skin_prices))) for (skin_count_var, skin_prices, _) in all_input_skins) == 10
model.total_skins_constraint = Constraint(rule=total_skins_constraint)

model.avg_input_float = Var(domain=NonNegativeReals, bounds=(0,1))
"""
Average float constraint
"""
def avg_float_constraint(model):
    return model.avg_input_float == sum(sum(skin_count_var[i] * skin_float[i] for i in range(len(skin_float))) for (skin_count_var, _, skin_float) in all_input_skins) / 10
    #return model.avg_input_float >= 0.0

model.avg_float_constraint = Constraint(rule=avg_float_constraint)


In [7]:
M = 10  # A large constant, big-M

"""
These constraints serve to select the correct binary variable out of the variables that represent each float of a skin
if a skin has 5 bounded floats variables, one for each condition (fn, mw, ft, ww, bs), and if the average float of the input skins is 0.03, then
the variable correspondent to 'fn' will have a value of 1, and the remaining four (mw, ft, ww, bs) variables will have a value of 0

We apply lower and upper constraints to account for the lower and upper limit of each condition class. For example, minimal wear is 0.07 to 0.15
The specific lower and upper threshold values are defined by each skin, given that these are influenced by the skin's min and max float.

For example, Ak-47 Redline's minimumm float is 0.1
Therefore, for the minimal wear variable, the lower threshold is 0.1 (opposed to 0.07), and the upper threshold is 0.15

The 'add_single_constraint' serves to make sure that for each skin, only one binary float variable is activated (the sum is 1). This is because there can't be
two conditions of the same skin in the output of a tradeup.
"""
def add_bound_constraints():
    # outcome skin float = ((max.float-min.float) × avg.float) + min.float
    for (output_var, _, _, skin_bounded_floats) in all_output_skins:
        min_float = skin_bounded_floats[0][0]
        max_float = skin_bounded_floats[-1][1]
        print(min_float)
        print(max_float)
        output_float = ((max_float - min_float) * model.avg_input_float) + min_float
        for i in range(len(skin_bounded_floats)):
            #lower_constraint = model.avg_input_float >= skin_bounded_floats[i][0] - M * (1 - output_var[i])
            #upper_constraint = model.avg_input_float <= skin_bounded_floats[i][1] + M * (1 - output_var[i])
            lower_constraint = output_float >= skin_bounded_floats[i][0] - M * (1 - output_var[i])
            upper_constraint = output_float <= skin_bounded_floats[i][1] + M * (1 - output_var[i])

            model.lower_bound_constraint.add(lower_constraint)
            model.upper_bound_constraint.add(upper_constraint)

def add_single_constraint():
    for (output_var, _, _, skin_bounded_floats) in all_output_skins:
        constraint = sum(output_var[i] for i in range(len(skin_bounded_floats))) == 1
        model.single_output_constraint.add(constraint)

model.lower_bound_constraint = ConstraintList()
model.upper_bound_constraint = ConstraintList()
model.single_output_constraint = ConstraintList()
add_bound_constraints()
add_single_constraint()

0
0.6
0
0.25
0
1
0
0.5
0
0.7
0
1
0
1
0
0.4
0.08
0.5
0
0.4
0.06
0.8
0
0.4


In [8]:

model.total_ballots = Var(domain=NonNegativeIntegers, bounds=(10, 80))

collection_to_vars_dict = {}

collection_num = 0
for (collection_name, collection) in list(collection_dict.items()):
    (input_skins, output_skins) = collection

    # set the variable that holds the number of ballots of the current collection
    #collection_ballots_var = Var(within=NonNegativeIntegers)
    collection_ballots_var = len(output_skins) * sum(sum(skin_count_var[i] for i in skin_count_var) for (skin_count_var,_,_) in input_skins) # Each input skin is a tuple (skin_count_var, skin_prices, skin_floats)
    setattr(model, f"collection_{collection_num}_ballots", collection_ballots_var)

    # set the variable which holds the probabilities of the outpt skins of this collectin
    collection_output_skins_names = [skin[1] for skin in output_skins] # skin name is in index 1 of the tuple (output_var, skin_name, skin_prices, skin_bounded_floats)
    collection_probs_var = Var(collection_output_skins_names, within=NonNegativeReals, bounds=(0,1))
    setattr(model, f"collection_{collection_num}_probs", collection_probs_var)
    
    # map the current collection to its variables, to set the constraints later
    collection_to_vars_dict[collection_name] = (collection_ballots_var, collection_probs_var)

    # create the collection count
    collection_num += 1

# Constraint for total_ballots
def total_ballots_rule(model):
    # total ballots is the sum of the ballots variable of each collection
    return model.total_ballots == sum(ballots_var for (ballots_var,_) in list(collection_to_vars_dict.values()))
model.total_ballots_constraint = Constraint(rule=total_ballots_rule)

def avg_output_price_rule(model):
    # skin prices for each collection. For example [skin_prices_collectionA, skin_prices_collectionB, ...]
    estimated_output_price = 0
    for (collection_name, collection) in list(collection_dict.items()):
        (_, output_skins) = collection
        (_, collection_probs_var) = collection_to_vars_dict[collection_name]
        for skin in output_skins:
            # each output_skin is a tuple (output_var, skin_name, skin_prices, skin_bounded_floats)
            output_var = skin[0] # output_var is at index 0
            skin_name = skin[1] # skin_name is at index 1
            prices_per_float = skin[2] # skin_prices is at index 2
            sum_prices = sum(output_var[i] * prices_per_float[i] for i in output_var) # get the skin price with the corresponding float
            # we can index collection_probs_var by skin_name
            # we multiply the price of the skin by its probability
            estimated_output_price += collection_probs_var[skin_name] * sum_prices 

    return model.average_output_price == estimated_output_price
model.avg_output_price_constraint = Constraint(rule=avg_output_price_rule)

model.ballots_per_collection_constraint = ConstraintList()
def add_ballots_per_collection_constraints():
    for (collection_name, collection) in list(collection_dict.items()):
        (_, output_skins) = collection
        (collection_ballots_var, collection_probs_var) = collection_to_vars_dict[collection_name]
        for skin in output_skins:
            skin_name = skin[1] # skin_name is at index 1
            constraint = collection_ballots_var == collection_probs_var[skin_name] * len(output_skins) * model.total_ballots
            model.ballots_per_collection_constraint.add(constraint)
add_ballots_per_collection_constraints()

In [9]:
# Choose a solver available in your environment, e.g., glpk, cbc, or ipopt for non-linear problems
#solver = SolverFactory('glpk')
# Check out AMPL for solvers
# user NEOS server maybe for solver
# also try apopt which is MINLP
# bonmin seems to work but is only for convex problems MINLP. non-convex finds only local solution
solver = SolverFactory('bonmin')
result = solver.solve(model, tee=True)

# Display results
model.pprint()

# Print the optimized values of the variables
for v in model.component_objects(Var, active=True):
    print("Variable", v)
    varobject = getattr(model, str(v))
    for index in varobject:
        print(" ", index, varobject[index].value)


Bonmin 1.8.9 using Cbc 2.10.8 and Ipopt 3.12.13
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -3.6809426       60 0.206
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT -1.0901452       54 0.149
NLP0014I             2         OPT -1.0901451       26 0.053
NLP0014I             3         OPT -1.071949       26 0.046
NLP0014I             4         OPT -1.0901452       56 0.156
NLP0014I             5         OPT -1.0901452      